# Handling Ambiguity and Improving Clarity in Prompt Engineering

## Overview

This tutorial focuses on two critical aspects of prompt engineering: identifying and resolving ambiguous prompts, and techniques for writing clearer prompts. These skills are essential for effective communication with AI models and obtaining more accurate and relevant responses.

## Motivation

Ambiguity in prompts can lead to inconsistent or irrelevant AI responses, while lack of clarity can result in misunderstandings and inaccurate outputs. By mastering these aspects of prompt engineering, you can significantly improve the quality and reliability of AI-generated content across various applications.

## Key Components

1. Identifying ambiguous prompts
2. Strategies for resolving ambiguity
3. Techniques for writing clearer prompts
4. Practical examples and exercises

## Method Details

We'll use OpenAI's GPT or any model  model and the LangChain library to demonstrate various techniques for handling ambiguity and improving clarity in prompts. The tutorial will cover:

1. Setting up the environment and necessary libraries
2. Analyzing ambiguous prompts and their potential interpretations
3. Implementing strategies to resolve ambiguity, such as providing context and specifying parameters
4. Exploring techniques for writing clearer prompts, including using specific language and structured formats
5. Practical exercises to apply these concepts in real-world scenarios

## Conclusion

By the end of this tutorial, you'll have a solid understanding of how to identify and resolve ambiguity in prompts, as well as techniques for crafting clearer prompts. These skills will enable you to communicate more effectively with AI models, resulting in more accurate and relevant outputs across various applications.

## Setup

First, let's import the necessary libraries and set up our environment.

In [56]:
# import os
# from langchain_openai import ChatOpenAI
# from langchain.prompts import PromptTemplate

# # Load environment variables
# from dotenv import load_dotenv
# load_dotenv()

# # Set up OpenAI API key
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# # Initialize the language model
# llm = ChatOpenAI(model="gpt-4o-mini")

- OpenAI (paid)

- GeminAI (free for limited time)

- We can use llama model using GroQ key

In [ ]:
groq_api="API_KEY"

## temperature (float, 0–2, default = 1.0)

- Controls randomness of the output.

- Lower = more deterministic, higher = more creative.

- 0.0 → nearly deterministic

- 0.7 → balanced (often used)

- >1.2 → very diverse/creative but may lose coherence

## max_tokens (int)

- Sets the maximum number of tokens in the generated response.

- Doesn’t affect input length — only the completion output.

- Example: max_tokens=512 will never generate more than 512 tokens.

## top_p (float, 0–1, default = 1.0)

- Enables nucleus sampling.

- Instead of sampling from all tokens, the model only considers the smallest set of tokens whose cumulative probability ≥ top_p.

- Lower values = more focused, higher values = more diverse.

- 0.9 → common choice, cuts off rare low-probability words

## top_k (int, default varies by model)

- Enables top-k sampling.

- The model only considers the k most likely next tokens before sampling.

- top_k=1 → greedy decoding (always picks most likely token)

- top_k=50 → sample from top 50 candidates

- Can be used together with top_p (model applies both filters).

## llama-3.3-70b-versatile

**Context Window & Output Length**

- According to Groq’s documentation, the model supports a context window of 131,072 tokens, which means it can process this much input (including prompt and response) in one go 

- The maximum output tokens (completion length) is 32,768 tokens 


- So effectively:

    - Max Input (prompt + response): 131,072 tokens

    - Max Output (completion): 32,768 tokens

- Vocabulary Size

| Specification         | Value                                                  |
| --------------------- | ------------------------------------------------------ |
| **Context Window**    | 131,072 tokens                                         |
| **Max Output Tokens** | 32,768 tokens                                          |
| **Vocabulary Size**   | Not officially stated; Llama 3.1 used \~128,000 tokens |


In [37]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key=groq_api,
    model="llama-3.3-70b-versatile",
    temperature=0.7,
    max_tokens=512
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002002F2E4590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002002F2E4FB0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=512)

## Identifying Ambiguous Prompts

Let's start by examining some ambiguous prompts and analyzing their potential interpretations.

In [39]:
ambiguous_prompts = [
    "Tell me about the bank.",
    "What's the best way to get to school?",
    "Can you explain the theory?"
]

for prompt in ambiguous_prompts:
    analysis_prompt = f"Analyze the following prompt for ambiguity: '{prompt}'. Explain why it's ambiguous and list possible interpretations."
    print(f"Prompt: {prompt}")
    print(llm.invoke(analysis_prompt).content)
    print("-" * 50)

Prompt: Tell me about the bank.
The prompt "Tell me about the bank" is ambiguous because the word "bank" has multiple meanings, and the context is not specified. This leads to several possible interpretations:

1. **Financial institution**: The most common association with "bank" is a financial institution where people deposit and withdraw money, such as a commercial bank or a central bank. In this context, the prompt might be asking for information about a specific bank's services, history, or location.
2. **Riverbank or lake bank**: A "bank" can also refer to the land alongside a river, lake, or sea. The prompt might be asking about the geography, geology, or features of a specific riverbank or lake bank.
3. **Storage location**: In some contexts, a "bank" can refer to a storage location for specific items, such as a blood bank (for storing blood donations) or a seed bank (for storing plant seeds).
4. **Slope or incline**: In aviation, a "bank" refers to the tilt of an aircraft durin

## Resolving Ambiguity

Now, let's explore strategies for resolving ambiguity in prompts.

In [41]:
def resolve_ambiguity(prompt, context):
    """
    Resolve ambiguity in a prompt by providing additional context.
    """
    clarified_prompt = f"{context}\n\nBased on this context, {prompt}"
    return llm.invoke(clarified_prompt).content

# Example usage
ambiguous_prompt = "Tell me about the bank."
contexts = [
    "You are a financial advisor discussing savings accounts.",
    "You are a geographer describing river formations."
]

for context in contexts:
    print(f"Context: {context}")
    print(f"Clarified response: {resolve_ambiguity(ambiguous_prompt, context)}")
    print("-" * 50)

Context: You are a financial advisor discussing savings accounts.
Clarified response: The bank I'm affiliated with is a well-established and reputable institution that offers a wide range of financial services, including savings accounts. We have a long history of providing our customers with secure and convenient ways to manage their money.

Our bank is committed to helping individuals and families achieve their financial goals, whether that's saving for a specific purpose, such as a down payment on a house, or simply building up an emergency fund. We offer a variety of savings account options, each with its own unique features and benefits.

Some of the key features of our savings accounts include competitive interest rates, low or no monthly maintenance fees, and easy access to your money through online banking, mobile banking, or in-person at one of our many branch locations. We also offer overdraft protection, which can help you avoid unexpected overdraft fees.

In addition to our

## Techniques for Writing Clearer Prompts

Let's explore some techniques for writing clearer prompts to improve AI responses.

In [43]:
def compare_prompt_clarity(original_prompt, improved_prompt):
    """
    Compare the responses to an original prompt and an improved, clearer version.
    """
    original_response = llm.invoke(original_prompt).content
    improved_response = llm.invoke(improved_prompt).content
    return original_response, improved_response

# Example usage
original_prompt = "How do I make it?"
improved_prompt = "Provide a step-by-step guide for making a classic margherita pizza, including ingredients and cooking instructions."

original_response, improved_response = compare_prompt_clarity(original_prompt, improved_prompt)

print("Original Prompt Response:")
print(original_response)
print("\nImproved Prompt Response:")
print(improved_response)

Original Prompt Response:
It seems like you're asking for instructions on making something, but you haven't specified what that something is. Could you please provide more details or clarify what you're trying to make? That way, I can give you more accurate and helpful guidance.

Improved Prompt Response:
The classic margherita pizza, a timeless Italian favorite that never fails to impress. Here's a step-by-step guide to making a delicious margherita pizza at home:

**Ingredients:**

For the dough:

* 1 1/2 cups warm water
* 1 tablespoon sugar
* 2 teaspoons active dry yeast
* 3 1/2 cups all-purpose flour
* 1 teaspoon salt
* 2 tablespoons olive oil

For the sauce:

* 2 cups San Marzano tomatoes, crushed by hand
* 2 cloves garlic, minced
* 1 tablespoon olive oil
* Salt, to taste

For the toppings:

* 8 ounces fresh mozzarella cheese, sliced
* Fresh basil leaves, chopped

**Instructions:**

**Step 1: Make the Dough (1 hour 30 minutes)**

1. In a large mixing bowl, combine the warm water, 

## Structured Prompts for Clarity

Using structured prompts can significantly improve clarity and consistency in AI responses.

In [49]:
from langchain.prompts import PromptTemplate

# Define a template with placeholders
template = """
You are an expert writing assistant.
Summarize the following text in 3 bullet points:

{text}
"""

# Create a PromptTemplate object
prompt = PromptTemplate(
    input_variables=["text"],  # variables you’ll pass in
    template=template
)

# Fill in the template
formatted_prompt = prompt.format(
    text="LangChain makes it easier to build LLM-powered apps."
)

print(formatted_prompt)

# 4. Send to LLM
response = llm.invoke(formatted_prompt)

# 5. Print model’s answer
print("Model output:\n", response.content)


You are an expert writing assistant.
Summarize the following text in 3 bullet points:

LangChain makes it easier to build LLM-powered apps.

Model output:
 Here are three bullet points summarizing the text:

* LangChain is a tool designed to simplify the development of applications.
* It specifically focuses on building apps that utilize Large Language Models (LLMs).
* By using LangChain, developers can more easily create and integrate LLM-powered functionality into their applications.


## Practical Exercise: Improving Prompt Clarity

Now, let's practice improving the clarity of prompts.

In [47]:
unclear_prompts = [
    "What's the difference?",
    "How does it work?",
    "Why is it important?"
]

def improve_prompt_clarity(unclear_prompt):
    """
    Improve the clarity of a given prompt.
    
    Args:
    unclear_prompt (str): The original unclear prompt
    
    Returns:
    str: An improved, clearer version of the prompt
    """
    improvement_prompt = f"The following prompt is unclear: '{unclear_prompt}'. Please provide a clearer, more specific version of this prompt. output just the improved prompt and nothing else." 
    return llm.invoke(improvement_prompt).content

for prompt in unclear_prompts:
    improved_prompt = improve_prompt_clarity(prompt)
    print(f"Original: {prompt}")
    print(f"Improved: {improved_prompt}")
    print("-" * 50)

Original: What's the difference?
Improved: What is the difference between two specific things, such as ideas, concepts, objects, or situations, and how do their characteristics, functions, or outcomes compare and contrast with one another?
--------------------------------------------------
Original: How does it work?
Improved: What are the specific mechanisms or processes by which this particular system, technology, or concept operates, and how do its various components interact to produce the desired outcome?
--------------------------------------------------
Original: Why is it important?
Improved: What are the significant benefits or consequences of this issue, and how does it impact individuals, communities, or society as a whole?
--------------------------------------------------
